In [1]:
#pip install tensorflow-addons

In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers


In [2]:
num_classes = 10
input_shape = (32, 32, 3)

AUTO=tf.data.AUTOTUNE
learning_rate = 0.001
batch_size = 128#256
hidden_units = 512
projection_units = 128
num_epochs = 50
dropout_rate = 0.5
temperature = 0.05



# Load the train and test data splits
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Display shapes of train and test datasets
print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

test_dataset=(tf.data.Dataset.from_tensor_slices((x_test, y_test))
    .shuffle(1024)
    .batch(batch_size)
    .prefetch(AUTO)
)





x_train shape: (50000, 32, 32, 3) - y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3) - y_test shape: (10000, 1)


In [3]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.02),
        layers.RandomWidth(0.2),
        layers.RandomHeight(0.2),
    ]
)

# Setting the state of the normalization layer.
data_augmentation.layers[0].adapt(x_train)


In [4]:
def create_encoder(backbone=keras.applications.ResNet50V2(
        include_top=False, weights=None, input_shape=input_shape, pooling="avg"
    )):
    

    inputs = keras.Input(shape=input_shape)
    augmented = data_augmentation(inputs)
    outputs = backbone(augmented)
    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-encoder")
    return model


encoder = create_encoder()
encoder.summary()



Model: "cifar10-encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 sequential (Sequential)     (None, 32, 32, 3)         7         
                                                                 
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
Total params: 23,564,807
Trainable params: 23,519,360
Non-trainable params: 45,447
_________________________________________________________________


In [5]:
def create_classifier(encoder, trainable=True):

    for layer in encoder.layers:
        layer.trainable = trainable

    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    features = layers.Dropout(dropout_rate)(features)
    features = layers.Dense(hidden_units, activation="relu")(features)
    features = layers.Dropout(dropout_rate)(features)
    outputs = layers.Dense(num_classes, activation="softmax")(features)

    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-classifier")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    return model


## model definition with the api

In [6]:

def apply_resnet_block(x,downsample,conv_by_block):
    
    depth_input=x.shape[-1]
    
    
    if downsample:
        depth=depth_input*2
        skiped=layers.Conv2D(depth,1,strides=(2,2),activation=None)(x)#linear projection
        x=layers.Conv2D(depth,3,strides=(2,2), activation='relu',padding="same")(x)
        x=layers.BatchNormalization()(x)
    else:
        depth=depth_input
        skiped=x
        x=layers.Conv2D(depth,3, activation='relu',padding="same")(x)
        x=layers.BatchNormalization()(x)
        
    for i in range(1,conv_by_block-1):
        x=layers.Conv2D(depth,3, activation='relu',padding="same")(x)
        x=layers.BatchNormalization()(x)
    
    x=layers.Conv2D(depth,3,padding="same")(x)#don't apply activation to the last 
    x=skiped+x
    x=layers.ReLU()(x)
    x=layers.BatchNormalization()(x)
   
    return x

def get_resnet_backbone(input_shape,hparams):
    """
    """
    assert hparams["conv_by_block"]>=2
    inputs=layers.Input((32,32,3))
    x=layers.Conv2D(hparams["depth_first_convolution"],7,strides=(2,2),activation='relu',padding="same")(inputs)
    x=layers.BatchNormalization()(x)
    for block in range(1,hparams["number_of_block"]+1):
        x=apply_resnet_block(x,block in hparams["downsample_num"],hparams["conv_by_block"])
       
        
    x=layers.Conv2D(hparams["output_dim"],3, activation='relu',padding="same")(x)
    x=layers.BatchNormalization()(x)
    if hparams["globalPoolingType"]=="Mean":
        
        x=layers.GlobalAveragePooling2D()(x)
 
    return tf.keras.Model(inputs,x)



hparams_resnet={"depth_first_convolution":64,
                "output_dim":2048,
                "number_of_block":5,
                "downsample_num":[4,5],
                "conv_by_block":2,
                "globalPoolingType":"Mean"
            }




resnet=get_resnet_backbone(input_shape,hparams_resnet)

resnet.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 16, 16, 64)   9472        ['input_3[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 16, 16, 64)  256         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 conv2d_1 (Conv2D)              (None, 16, 16, 64)   36928       ['batch_normalization[0][0]']

                                                                                                  
 re_lu_4 (ReLU)                 (None, 4, 4, 256)    0           ['tf.__operators__.add_4[0][0]'] 
                                                                                                  
 batch_normalization_10 (BatchN  (None, 4, 4, 256)   1024        ['re_lu_4[0][0]']                
 ormalization)                                                                                    
                                                                                                  
 conv2d_13 (Conv2D)             (None, 4, 4, 2048)   4720640     ['batch_normalization_10[0][0]'] 
                                                                                                  
 batch_normalization_11 (BatchN  (None, 4, 4, 2048)  8192        ['conv2d_13[0][0]']              
 ormalization)                                                                                    
          

In [68]:
import time as time

class SupervisedContrastiveLoss(keras.losses.Loss):
    def __init__(self, temperature=1, name=None):
        super(SupervisedContrastiveLoss, self).__init__(reduction=tf.keras.losses.Reduction.NONE,name=name)
        self.temperature = temperature

    def __call__(self, y_true, feature_vectors,sample_weight=None):
        #adapted from https://github.com/HobbitLong/SupContrast/blob/master/losses.py
        #only change should be the change of computation of the maximum for the numerical stability trick
        
        feature_vectors1,feature_vectors2=feature_vectors[0],feature_vectors[1]
        
        #feature vectors1_2 : shape (batch_size,_)
        feature_vectors=tf.concat([feature_vectors1,feature_vectors2],axis=0)
        # Normalize feature vectors
        feature_vectors_normalized = tf.math.l2_normalize(feature_vectors, axis=1)
        # Compute logits
        logits = tf.divide(
            tf.matmul(
                feature_vectors_normalized, tf.transpose(feature_vectors_normalized)
            ),
            self.temperature,
        )
        
        
        #y_pred = tf.convert_to_tensor(logits)
        #print(y_pred.shape)
        y_true = tf.cast(y_true, logits.dtype)

        # Expand to [2*batch_size, 1]
        
        
        
        ones_bs=tf.ones_like(y_true)[:,0]#zeros of size (batch_size)
        
        
        vector_bs_bs=tf.cast(tf.equal(y_true, tf.transpose(y_true)), logits.dtype)
        
        id_vector=tf.zeros_like(vector_bs_bs)#zero of size (batch_size,batch_size)
        id_vector=tf.linalg.set_diag(id_vector,ones_bs)#set the diagonal to ones
        
        
        is_similar=tf.concat([tf.concat([id_vector,id_vector],axis=0),tf.concat([id_vector,id_vector],axis=0)],axis=1)
        
        #y_true=tf.concat([y_true,y_true],axis=0)
        #y_true = tf.expand_dims(y_true, -1)#depend on the input shape of the labels
        #is_similar = tf.cast(tf.equal(y_true, tf.transpose(y_true)), logits.dtype)#for supervised learning
        
        
        #mask for self_contrasting terms
       
        t=time.time()
        
        mask_logits=tf.ones_like(logits)#one of size (2*batch_size,2*batch_size)
        zeros_bs2=tf.zeros_like(logits)[0]#zeros of size (2*batch_size)
        mask_logits=tf.linalg.set_diag(mask_logits,zeros_bs2)#set the diagonal to 0
        
        #for numerical stability 

        max_logits=tf.math.reduce_max(mask_logits*logits,axis=1,keepdims=True)#take the max possible value for the logits
        logits=logits-tf.stop_gradient(max_logits)
        
        #computation of the loss
        
        print(is_similar)
        print(mask_logits)
        is_similar=is_similar*mask_logits# do not include the contrastive loss between 2 sames images
        
        log_softmax=logits-tf.math.log(tf.reduce_sum(mask_logits*tf.math.exp(logits),axis=1,keepdims=True))
        
        loss=tf.reduce_sum(is_similar*log_softmax,axis=1)/tf.reduce_sum(is_similar,axis=1)#compute the mean of the log likelhood
        
        return -tf.math.reduce_mean(loss)
                

def add_projection_heads(encoder):
    inputs = keras.Input(shape=input_shape)
    features1 = encoder(inputs)
    features2 = encoder(inputs)
    outputs1 = layers.Dense(projection_units, activation="relu")(features1)
    outputs2 = layers.Dense(projection_units, activation="relu")(features2)
    model = keras.Model(
        inputs=inputs, outputs=tf.stack([outputs1,outputs2]), name="cifar-encoder_with_projection-head"
    )
    return model


In [69]:
early_stop_unsup=tf.keras.callbacks.EarlyStopping(
    monitor="loss", patience=3, restore_best_weights=True
)
learning_rate=0.001
resnet=get_resnet_backbone(input_shape,hparams_resnet)

encoder = create_encoder(resnet)

encoder_with_projection_head = add_projection_heads(encoder)
encoder_with_projection_head.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=SupervisedContrastiveLoss(temperature),
)

encoder_with_projection_head.summary()

history = encoder_with_projection_head.fit(
    x=x_train, y=y_train, batch_size=batch_size, epochs=25,callbacks=[early_stop_unsup]
)

Model: "cifar-encoder_with_projection-head"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_94 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 cifar10-encoder (Functional)   (None, 2048)         6112775     ['input_94[0][0]',               
                                                                  'input_94[0][0]']               
                                                                                                  
 dense_60 (Dense)               (None, 128)          262272      ['cifar10-encoder[0][0]']        
                                                                                                  
 dense_61 (Dense)               (None, 128)          262272      

KeyboardInterrupt: 

In [70]:
early_stop=tf.keras.callbacks.EarlyStopping(
    monitor="sparse_categorical_accuracy", patience=3, restore_best_weights=True
)


classifier = create_classifier(encoder, trainable=False)


history = classifier.fit(x=x_train, y=y_train,validation_data=test_dataset, batch_size=batch_size, epochs=num_epochs,callbacks=[early_stop])

accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")


Epoch 1/50
391/391 [==============================] - 18s 39ms/step - loss: 2.4012 - sparse_categorical_accuracy: 0.2514 - val_loss: 1.8333 - val_sparse_categorical_accuracy: 0.3371
Epoch 2/50
391/391 [==============================] - 14s 37ms/step - loss: 1.9418 - sparse_categorical_accuracy: 0.2961 - val_loss: 1.7985 - val_sparse_categorical_accuracy: 0.3516
Epoch 3/50
391/391 [==============================] - 15s 39ms/step - loss: 1.8885 - sparse_categorical_accuracy: 0.3113 - val_loss: 1.7833 - val_sparse_categorical_accuracy: 0.3568
Epoch 4/50
391/391 [==============================] - 15s 39ms/step - loss: 1.8797 - sparse_categorical_accuracy: 0.3157 - val_loss: 1.7831 - val_sparse_categorical_accuracy: 0.3523
Epoch 5/50
391/391 [==============================] - 4569s 12s/step - loss: 1.8784 - sparse_categorical_accuracy: 0.3178 - val_loss: 1.7852 - val_sparse_categorical_accuracy: 0.3525
Epoch 6/50
391/391 [==============================] - 63s 159ms/step - loss: 1.8768 - spa